In [1]:
# Reed Whitcraft
# Mod 02 Final Project

# Table of Contents
***

    1. Observation
    2. Examine Research
    3. Form Hypothesis
    4. Conduct Experiment
    5. Analyze Experimental Results
    6. Draw Conclusions

# Lab Links
***
- [Querying with SQLAlchemy](https://learn.co/tracks/data-science-career-v1-1/module-2-advanced-data-retrieval-and-analysis/section-15-an-introduction-to-orms/querying-with-sqlalchemy)
- [Querying with SQLAlchemy - Lab (Solution)](https://github.com/learn-co-students/dsc-2-13-11-queries-with-sqlalchemy-lab-online-ds-ft-021119/tree/solution)
- [Using an ORM](https://learn.co/tracks/data-science-career-v1-1/module-2-advanced-data-retrieval-and-analysis/section-15-an-introduction-to-orms/using-an-orm)
- [Using Sqlite - Lab (Solution)](https://github.com/learn-co-students/dsc-2-13-04-review-using-sqlite-lab-online-ds-ft-021119/tree/solution)


# Database Details

<table>
    <thead>
        <tr>
            <th>Table Name</th>
            <th>Primary Key</th>
            <th>Foreign Key(s)</th>
            <th>Count</th>
        </tr>
    </thead>
    <tbody>
        <tr><td>Product</td><td>d</td><td>d</td><td>77</td></tr>
        <tr><td>Category</td><td>d</td><td>d</td><td>8</td></tr>
        <tr><td>Order</td><td>d</td><td>d</td><td>830</td></tr>
        <tr><td>OrderDetail</td><td>d</td><td>d</td><td>2155</td></tr>
        <tr><td>Customer</td><td>d</td><td>d</td><td>91</td></tr>
    </tbody>
</table>

In [62]:
# import dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import inspect

In [63]:
# connecting to database
engine = create_engine("sqlite:///Northwind_small.sqlite", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
inspector = inspect(engine)
print(inspector.get_table_names()) # print table names

2019-03-31 18:15:55,105 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-31 18:15:55,105 INFO sqlalchemy.engine.base.Engine ()
2019-03-31 18:15:55,107 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-31 18:15:55,108 INFO sqlalchemy.engine.base.Engine ()
2019-03-31 18:15:55,110 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-31 18:15:55,111 INFO sqlalchemy.engine.base.Engine ()
['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [64]:
# helper methods

def get_columns_info(col_name):
    cols_list = inspector.get_columns(col_name)
    print("Table Name: {}".format(col_name))
    print("")
    for column in cols_list:
        print("Name: {} \t\t Type: {}".format(column['name'], column['type']))

In [86]:
get_columns_info('Order')

2019-04-01 13:32:33,614 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Order")
2019-04-01 13:32:33,615 INFO sqlalchemy.engine.base.Engine ()
Table Name: Order

Name: Id 		 Type: INTEGER
Name: CustomerId 		 Type: VARCHAR(8000)
Name: EmployeeId 		 Type: INTEGER
Name: OrderDate 		 Type: VARCHAR(8000)
Name: RequiredDate 		 Type: VARCHAR(8000)
Name: ShippedDate 		 Type: VARCHAR(8000)
Name: ShipVia 		 Type: INTEGER
Name: Freight 		 Type: DECIMAL
Name: ShipName 		 Type: VARCHAR(8000)
Name: ShipAddress 		 Type: VARCHAR(8000)
Name: ShipCity 		 Type: VARCHAR(8000)
Name: ShipRegion 		 Type: VARCHAR(8000)
Name: ShipPostalCode 		 Type: VARCHAR(8000)
Name: ShipCountry 		 Type: VARCHAR(8000)


In [67]:
get_columns_info('Orderdetail')

2019-03-31 19:50:19,138 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Orderdetail")
2019-03-31 19:50:19,139 INFO sqlalchemy.engine.base.Engine ()
Table Name: Orderdetail

Name: Id 		 Type: VARCHAR(8000)
Name: OrderId 		 Type: INTEGER
Name: ProductId 		 Type: INTEGER
Name: UnitPrice 		 Type: DECIMAL
Name: Quantity 		 Type: INTEGER
Name: Discount 		 Type: FLOAT


In [76]:
get_columns_info('Product')

2019-03-31 19:57:41,437 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Product")
2019-03-31 19:57:41,438 INFO sqlalchemy.engine.base.Engine ()
Table Name: Product

Name: Id 		 Type: INTEGER
Name: ProductName 		 Type: VARCHAR(8000)
Name: SupplierId 		 Type: INTEGER
Name: CategoryId 		 Type: INTEGER
Name: QuantityPerUnit 		 Type: VARCHAR(8000)
Name: UnitPrice 		 Type: DECIMAL
Name: UnitsInStock 		 Type: INTEGER
Name: UnitsOnOrder 		 Type: INTEGER
Name: ReorderLevel 		 Type: INTEGER
Name: Discontinued 		 Type: INTEGER


In [103]:
get_columns_info('Category')

2019-04-01 14:17:32,205 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Category")
2019-04-01 14:17:32,205 INFO sqlalchemy.engine.base.Engine ()
Table Name: Category

Name: Id 		 Type: INTEGER
Name: CategoryName 		 Type: VARCHAR(8000)
Name: Description 		 Type: VARCHAR(8000)


In [90]:
q = '''SELECT unitprice, quantity, discount from orderdetail'''
df = pd.read_sql_query(q, engine)
df.describe()

2019-04-01 13:41:47,908 INFO sqlalchemy.engine.base.Engine SELECT unitprice, quantity, discount from orderdetail
2019-04-01 13:41:47,909 INFO sqlalchemy.engine.base.Engine ()


,UnitPrice,Quantity,Discount
count,2155.000000,2155.000000,2155.000000
mean,26.218520,23.812993,0.056167
std,29.827418,19.022047,0.083450
min,2.000000,1.000000,0.000000
25%,12.000000,10.000000,0.000000
50%,18.400000,20.000000,0.000000
75%,32.000000,30.000000,0.100000
max,263.500000,130.000000,0.250000


In [91]:
df.Discount.value_counts()

0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Discount, dtype: int64

In [88]:
no_discount_q = '''SELECT SUM(quantity) as total_quantity, OrderId FROM Orderdetail JOIN [Order] on Orderdetail.OrderId = "Order".Id WHERE Orderdetail.discount > 0 GROUP BY Orderdetail.OrderId ORDER BY total_quantity DESC'''
df = pd.read_sql_query(no_discount_q, engine)
df.head()

2019-04-01 13:34:48,396 INFO sqlalchemy.engine.base.Engine SELECT SUM(quantity) as total_quantity, OrderId FROM Orderdetail JOIN [Order] on Orderdetail.OrderId = "Order".Id WHERE Orderdetail.discount > 0 GROUP BY Orderdetail.OrderId ORDER BY total_quantity DESC
2019-04-01 13:34:48,397 INFO sqlalchemy.engine.base.Engine ()


,total_quantity,OrderId
0,288,10847
1,260,11030
2,238,10451
3,215,10595
4,211,10324


In [97]:
q = '''SELECT * FROM orderdetail WHERE OrderId = 11030'''
df = pd.read_sql_query(q, engine)
df.Discount.value_counts()

2019-04-01 14:05:57,974 INFO sqlalchemy.engine.base.Engine SELECT * FROM orderdetail WHERE OrderId = 11030
2019-04-01 14:05:57,975 INFO sqlalchemy.engine.base.Engine ()


0.25    3
0.00    1
Name: Discount, dtype: int64

In [109]:
q = '''SELECT product.Id, product.ProductName, product.CategoryId, product.UnitPrice as ProductUnitPrice, Orderdetail.OrderId, Orderdetail.ProductId, Orderdetail.UnitPrice, Orderdetail.Quantity, Orderdetail.Discount FROM PRODUCT JOIN ORDERDETAIL ON product.Id = OrderDetail.ProductId'''
df = pd.read_sql_query(q, engine)
df.head()
df.describe()

2019-04-01 14:19:15,776 INFO sqlalchemy.engine.base.Engine SELECT product.Id, product.ProductName, product.CategoryId, product.UnitPrice as ProductUnitPrice, Orderdetail.OrderId, Orderdetail.ProductId, Orderdetail.UnitPrice, Orderdetail.Quantity, Orderdetail.Discount FROM PRODUCT JOIN ORDERDETAIL ON product.Id = OrderDetail.ProductId
2019-04-01 14:19:15,777 INFO sqlalchemy.engine.base.Engine ()


,Id,CategoryId,ProductUnitPrice,OrderId,ProductId,UnitPrice,Quantity,Discount
count,2155.000000,2155.000000,2155.000000,2155.000000,2155.000000,2155.000000,2155.000000,2155.000000
mean,40.793039,4.135499,27.948687,10659.375870,40.793039,26.218520,23.812993,0.056167
std,22.159019,2.378567,31.610706,241.378032,22.159019,29.827418,19.022047,0.083450
min,1.000000,1.000000,2.500000,10248.000000,1.000000,2.000000,1.000000,0.000000
25%,22.000000,2.000000,12.500000,10451.000000,22.000000,12.000000,10.000000,0.000000
50%,41.000000,4.000000,19.450000,10657.000000,41.000000,18.400000,20.000000,0.000000
75%,60.000000,6.000000,34.000000,10862.500000,60.000000,32.000000,30.000000,0.100000
max,77.000000,8.000000,263.500000,11077.000000,77.000000,263.500000,130.000000,0.250000


In [79]:
# df = pd.read_sql_query('''SELECT * FROM orderdetail GROUP BY OrderId''', engine)
# df = pd.read_sql_query('''SELECT * FROM "order"''', engine)
# df.head(30)
q = '''SELECT COUNT(ProductId) FROM Product JOIN Orderdetail on product.Id = Orderdetail.ProductId GROUP BY orderdetail.OrderId'''

df = pd.read_sql_query(q, engine)
df.head(50)

2019-03-31 19:58:26,827 INFO sqlalchemy.engine.base.Engine SELECT COUNT(ProductId) FROM Product JOIN Orderdetail on product.Id = Orderdetail.ProductId GROUP BY orderdetail.OrderId
2019-03-31 19:58:26,828 INFO sqlalchemy.engine.base.Engine ()


,COUNT(ProductId)
0,3
1,2
2,3
3,3
4,3
5,3
6,3
7,4
8,2
9,3


In [41]:
# conn = sqlite3.connect('Northwind_small.sqlite')
# c = conn.cursor()

# def query_all_table(table_name):
#     sql_string = "SELECT * FROM " + table_name
#     all_rows = c.execute(sql_string).fetchall()
#     print("The number of {} is {}".format(table_name, len(all_rows)))
#     return all_rows

# def query_and_fetch(sql_string):
#     return c.execute(sql_string).fetchall()

In [46]:
# products = query_all_table('product')
# categories = query_all_table('category')
# orders = query_all_table("'order'")
# order_details = query_all_table('orderdetail')
# customers = query_all_table('customer')

The number of product is 77
The number of category is 8
The number of 'order' is 830
The number of orderdetail is 2155
The number of customer is 91
